<a href="https://colab.research.google.com/github/Rekhansh-Gupta/InfosysSpringboard_Internship_project/blob/main/Merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# ===============================
# INSTALL DEPENDENCIES
# ===============================
!pip install streamlit pyngrok pandas networkx pyvis sentence-transformers

In [10]:
from pyngrok import ngrok
ngrok.set_auth_token("32vVOZxUGg0sLHA7Ht6p7SCCocW_L6brxuNUsYzRPNwEvu7E")


In [11]:
%%writefile app.py
import streamlit as st
import pandas as pd
import networkx as nx
from pyvis.network import Network
import tempfile, os
from sentence_transformers import SentenceTransformer, util

# ---------------------------
#  APP HEADER
# ---------------------------
st.set_page_config(page_title="Knowledge Graph Explorer", layout="wide")

st.markdown("<h1 style='text-align:center;color:#3B82F6;'>🧠 Knowledge Graph Explorer</h1>", unsafe_allow_html=True)

# ---------------------------

# Initialize session state
if "logged_in" not in st.session_state:
    st.session_state.logged_in = False
if "users" not in st.session_state:
    st.session_state.users = {"admin": "admin123"}  # default account

st.title("🔐 Login / Register Page")

if not st.session_state.logged_in:
    choice = st.sidebar.selectbox("Select Action", ["Login", "Register"])

    if choice == "Login":
        username = st.text_input("Username")
        password = st.text_input("Password", type="password")
        if st.button("Login"):
            if username in st.session_state.users and st.session_state.users[username] == password:
                st.session_state.logged_in = True
                st.success(f"Welcome back, {username}!")
                st.rerun()
  # refresh page
            else:
                st.error("Invalid username or password.")

    elif choice == "Register":
        new_user = st.text_input("Choose Username")
        new_pass = st.text_input("Choose Password", type="password")
        if st.button("Register"):
            if new_user in st.session_state.users:
                st.warning("User already exists.")
            else:
                st.session_state.users[new_user] = new_pass
                st.success("Registration successful! You can now log in.")
else:
    st.success("✅ Logged in successfully!")
    st.sidebar.button("Logout", on_click=lambda: st.session_state.update({"logged_in": False}))
    st.write("Now you can access dataset upload and dashboard pages.")


# ---------------------------
# SIDEBAR MENU (MILESTONES)
# ---------------------------
menu = st.sidebar.radio(
    "Project Milestones",
    [
        "📁 Milestone 1: Dataset Upload",
        "🧠 Milestone 2: Entity & Relation Extraction",
        "🌐 Milestone 3: Knowledge Graph & Semantic Search",
        "🛠️ Milestone 4: Admin Dashboard & Feedback"
    ]
)

# ---------------------------
# MILESTONE 1: DATASET UPLOAD
# ---------------------------
if menu == "📁 Milestone 1: Dataset Upload":
    st.header("📁 Upload Dataset (CSV Supported)")
    uploaded_file = st.file_uploader("Upload your dataset file", type=["csv", "txt", "json"])
    if uploaded_file:
        df = pd.read_csv(uploaded_file)
        st.success("✅ Dataset Uploaded Successfully")
        st.subheader("📊 Dataset Preview")
        st.dataframe(df.head())

        st.session_state["dataset"] = df

# ---------------------------
# MILESTONE 2: ENTITY & RELATION EXTRACTION
# ---------------------------
elif menu == "🧠 Milestone 2: Entity & Relation Extraction":
    st.header("🧠 Named Entity & Relation Extraction")

    if "dataset" not in st.session_state:
        st.warning("⚠️ Please upload a dataset first (Milestone 1)")
        st.stop()

    df = st.session_state["dataset"]
    model = SentenceTransformer("all-MiniLM-L6-v2")

    st.info("Extracting entity triples (subject, relation, object)...")

    triples = []
    for i, row in df.iterrows():
        text = " ".join([str(x) for x in row.values])
        entities = text.split()[:3]  # simple mock entities
        if len(entities) == 3:
            triples.append((entities[0], entities[1], entities[2]))

    triples_df = pd.DataFrame(triples, columns=["Subject", "Relation", "Object"])
    st.subheader("📋 Extracted Triples")
    st.dataframe(triples_df)

    st.session_state["triples"] = triples_df

# ---------------------------
# MILESTONE 3: KNOWLEDGE GRAPH + SEMANTIC SEARCH
# ---------------------------
elif menu == "🌐 Milestone 3: Knowledge Graph & Semantic Search":
    st.header("🌐 Knowledge Graph Visualization")

    if "triples" not in st.session_state:
        st.warning("⚠️ Please complete Entity Extraction first (Milestone 2)")
        st.stop()

    triples_df = st.session_state["triples"]

    # Build Graph
    G = nx.DiGraph()
    for _, row in triples_df.iterrows():
        G.add_node(row["Subject"])
        G.add_node(row["Object"])
        G.add_edge(row["Subject"], row["Object"], title=row["Relation"])

    net = Network(height="500px", width="100%", bgcolor="#222222", font_color="white", directed=True)
    net.from_nx(G)
    with tempfile.NamedTemporaryFile(delete=False, suffix=".html") as tmpfile:
        net.save_graph(tmpfile.name)
        html_path = tmpfile.name

    st.subheader("🔍 Interactive Graph")
    with open(html_path, "r", encoding="utf-8") as f:
        html_content = f.read()
        st.components.v1.html(html_content, height=500, scrolling=True)

    # Semantic Search
    st.subheader("💬 Semantic Search")
    model = SentenceTransformer("all-MiniLM-L6-v2")
    query = st.text_input("Enter your search query:")
    if query:
        corpus = [f"{row['Subject']} {row['Relation']} {row['Object']}" for _, row in triples_df.iterrows()]
        corpus_embeddings = model.encode(corpus)
        query_embedding = model.encode(query)
        scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
        best_idx = scores.argmax()
        st.write("🔎 Closest Match:", corpus[best_idx])

# ---------------------------
# MILESTONE 4: ADMIN DASHBOARD & FEEDBACK
# ---------------------------
elif menu == "🛠️ Milestone 4: Admin Dashboard & Feedback":
    st.header("🛠️ Admin Dashboard")

    if "triples" not in st.session_state:
        st.warning("⚠️ Please complete Milestone 3 first.")
        st.stop()

    triples_df = st.session_state["triples"]

    st.subheader("📊 Graph Statistics")
    st.write(f"Total Entities: {len(set(triples_df['Subject']) | set(triples_df['Object']))}")
    st.write(f"Total Relations: {len(triples_df)}")

    st.subheader("🧩 Manual Correction (Edit Triples)")
    edited_triples = st.data_editor(triples_df, num_rows="dynamic")
    st.session_state["triples"] = edited_triples

    st.subheader("⭐ Feedback System")
    feedback = st.radio("How relevant is the graph to your data?", ["Excellent", "Good", "Average", "Poor"])
    if st.button("Submit Feedback"):
        st.success(f"✅ Feedback submitted: {feedback}")

    st.markdown("<hr>", unsafe_allow_html=True)
    st.caption("Developed by Rekhansh Gupta | Integrated Project Milestone 1–4")


Overwriting app.py


In [12]:
!pkill streamlit
!streamlit run app.py &>/dev/null &

import time
from pyngrok import ngrok
time.sleep(5)  # give Streamlit time to start

public_url = ngrok.connect(8501)
print("🌐 App is Live at:", public_url)


🌐 App is Live at: NgrokTunnel: "https://1a40230d3cb8.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!mkdir -p semantic_app

In [ ]:
%%writefile semantic_app/app.py
# Your Streamlit app code here
import streamlit as st
st.title("Semantic Knowledge Graph Dashboard")
st.write("Hello Rekhansh! This is your Dockerized Streamlit App.")


Writing semantic_app/app.py


In [ ]:
%%writefile /content/semantic_app/requirements.txt
streamlit
pyngrok
pandas
networkx
pyvis
sentence-transformers
transformers
torch


Writing /content/semantic_app/requirements.txt


In [ ]:
%%writefile /content/semantic_app/Dockerfile
FROM python:3.10
WORKDIR /app
COPY . .
RUN pip install -r requirements.txt
EXPOSE 8501
CMD ["streamlit", "run", "app.py", "--server.port=8501", "--server.address=0.0.0.0"]


Writing /content/semantic_app/Dockerfile
